# SMART-G validation IPRT phase A 
- https://www.meteo.physik.uni-muenchen.de/~iprt/doku.php?id=start

In [ ]:
%matplotlib inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2

import os, sys

try:
    import subprocess
    check = subprocess.check_call(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    ROOTPATH = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8') # Root Git Path
except subprocess.CalledProcessError:
    ROOTPATH = os.getcwd()
sys.path.insert(0, ROOTPATH)

from smartg.config import DIR_AUXDATA
from smartg.smartg import Smartg, Sensor, RoughSurface, LambSurface, Albedo_cst
from smartg.atmosphere import AtmAFGL, diff1, trapzinterp

from luts.luts import LUT, Idx
import pandas as pd
import numpy as np
from pathlib import Path

from smartg.tools.phase import calc_iphase
import matplotlib.pyplot as plt

from smartg.iprt.iprt import convert_SGout_to_IPRTout, select_and_plot_polar_iprt, seclect_iprt_IQUV, compute_deltam, plot_iprt_radiances, groupIQUV, read_phase_nth_cte


output_folder_path = ROOTPATH + "/smartg/notebooks/SMARTG_RES_IPRT_REF/"
Path(output_folder_path).mkdir(parents=True, exist_ok=True)

OPT_PROP_PATH = DIR_AUXDATA + "/IPRT/phaseA/opt_prop/"
MYSTIC_RES_PATH = DIR_AUXDATA + "/IPRT/phaseA/mystic_res/"

# Compilation in forward and backward
S1DF = Smartg(alt_pp=False, back=False, double=True, bias=True)
S1DB = Smartg(alt_pp=False, back=True, double=True, bias=True)

SEED = 1e8

## Test cases including a single layer

### Case A1

#### Atmosphere profil

In [ ]:
mol_sca = np.array([0., 0.5])[None,:]
mol_abs= np.array([0., 0.])[None,:]
z = np.array([1., 0.])
wl = 550.

proA1 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs).calc(wl)
surfA1  = None
NBPH = 1e8

#### Compute radiances depol=0, sza=0, saa=65
- Note: In forward there is a bug for SZA near to 0 (see Q and U values), then simulation is performed in backward (more comsuming) 

In [ ]:
SZA = 0.
SAA = 65.
PHI_0 = 180.-SAA # To follow MYSTIC convention
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([PHI_0])}

# BOA radiances
VZAMIN = 0.
VZAMAX = 80.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

# VAA from 0. to 180.
VAAMIN = 0.
VAAMAX = 360.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

lsensors = []
NBVZA = len(VZA)
NBVAA = len(VAA)
NBDIR = round(NBVAA*NBVZA)
for iza, za in enumerate(VZA):
    for iaa, aa in enumerate(VAA):
        PHI = -aa+180
        lsensors.append(Sensor(POSZ=np.min(z), THDEG=za, PHDEG=PHI, LOC='ATMOS'))

mA1B = S1DB.run(wl=550., NBPHOTONS=NBPH*NBDIR, NBLOOP=1e8, atm=proA1, sensor=lsensors,
                le=le, surf=surfA1, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.0, stdev=True, progress=True, SEED=SEED)

mA1B = mA1B.dropaxis('Azimuth angles', 'Zenith angles')
mA1B.add_axis('Azimuth angles', -VAA+180.)
mA1B.add_axis('Zenith angles', VZA)

for name in mA1B.datasets():
    if 'sensor index' in mA1B[name].names:
        mat_tmp = np.swapaxes(mA1B[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mA1B[name].attrs
        mA1B.rm_lut(name)
        mA1B.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mA1B_boa_dep0 = mA1B.dropaxis('sensor index')

# TOA radiances
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

lsensors = []
NBVZA = len(VZA)
NBVAA = len(VAA)
NBDIR = round(NBVAA*NBVZA)
for iza, za in enumerate(VZA):
    for iaa, aa in enumerate(VAA):
        PHI = -aa+180
        lsensors.append(Sensor(POSZ=np.max(z), THDEG=za, PHDEG=PHI, LOC='ATMOS'))

mA1B = S1DB.run(wl=wl, NBPHOTONS=NBPH*NBDIR, NBLOOP=1e8, atm=proA1, sensor=lsensors,
                le=le, surf=surfA1, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.0, stdev=True, progress=True, SEED=SEED)

mA1B = mA1B.dropaxis('Azimuth angles', 'Zenith angles')
mA1B.add_axis('Azimuth angles', -VAA+180.)
mA1B.add_axis('Zenith angles', VZA)

for name in mA1B.datasets():
    if 'sensor index' in mA1B[name].names:
        mat_tmp = np.swapaxes(mA1B[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mA1B[name].attrs
        mA1B.rm_lut(name)
        mA1B.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mA1B_toa_dep0 = mA1B.dropaxis('sensor index')

In [ ]:
mA1B_boa_dep0.save(output_folder_path + "iprt_a1_smartg_dep0_boa_ref.nc", overwrite=True)
mA1B_toa_dep0.save(output_folder_path + "iprt_a1_smartg_dep0_toa_ref.nc", overwrite=True)

#### Compute radiances depol=0.03, sza=30, saa=0

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 360.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

SZA = 30.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mA1F_dep003 = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proA1, OUTPUT_LAYERS=int(1),
                       le=le, surf=surfA1, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mA1F_dep003.save(output_folder_path + "iprt_a1_smartg_dep003_ref.nc", overwrite=True)

#### Compute radiances depol=0.1, sza=30, saa=65

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 360.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

SZA = 30.0
SAA = 65.0
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

# stdev_lim = StdevLim(err_rel_min=1e-2, format=".2e", verbose=True)
mA1F_dep01 = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proA1, OUTPUT_LAYERS=int(1),
                       le=le, surf=surfA1, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.1, stdev=True, SEED=SEED)

In [ ]:
mA1F_dep01.save(output_folder_path + "iprt_a1_smartg_dep01_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances 
VZA_boa_dep0 = mA1B_boa_dep0.axes['Zenith angles']
VAA_boa_dep0 = 180.-mA1B_boa_dep0.axes['Azimuth angles']
VZA_toa_dep0 = mA1B_toa_dep0.axes['Zenith angles']
VAA_toa_dep0 = 180.-mA1B_toa_dep0.axes['Azimuth angles']

VZA_dep003 = 180.-mA1F_dep003.axes['Zenith angles']
VAA_dep003 = -mA1F_dep003.axes['Azimuth angles']
VZA_dep01 = 180.-mA1F_dep01.axes['Zenith angles']
VAA_dep01 = -mA1F_dep01.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a1_smartg_ref.dat"
case_name = "A1"
ldepol    = [0., 0., 0.03, 0.03, 0.1, 0.1]
lalt      = [0., 1., 0., 1., 0., 1.]
lSZA      = [0., 0., 30., 30., 30., 30.]
lSAA      = [65., 65., 0., 0., 65., 65.]

# convert
convert_SGout_to_IPRTout(lm=[mA1B_boa_dep0, mA1B_toa_dep0, mA1F_dep003, mA1F_dep003, mA1F_dep01, mA1F_dep01], lU_sign=[1., 1, -1, -1, -1, -1], case_name=case_name,
                         ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,lVZA=[VZA_boa_dep0, VZA_toa_dep0, 180.-VZA_dep003, VZA_dep003, 180.-VZA_dep01, VZA_dep01],
                         lVAA=[VAA_boa_dep0, VAA_toa_dep0, VAA_dep003, VAA_dep003, VAA_dep01, VAA_dep01], file_name=filename,
                         output_layer=['_up (TOA)', '_up (TOA)', '_down (0+)', '_up (TOA)', '_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename  = output_folder_path + "iprt_case_a1_smartg_ref.dat"
case_name = "A1"
ldepol    = [0., 0., 0.03, 0.03, 0.1, 0.1]
lalt      = [0., 1., 0., 1., 0., 1.]
lSZA      = [0., 0., 30., 30., 30., 30.]
lSAA      = [65., 65., 0., 0., 65., 65.]
lINVTH    = [False, True, False, True, False, True]

smartg_a1 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a1 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a1_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_a1
ref_model = mystic_a1
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True, False, True, False, True]
lINVTH_mod   = [False, True, False, True, False, True]
lU_sign_rmod  =[True, True, True, True, True, True]
lU_sign_mod  = [True, True, True, True, True, True]
lV_sign_rmod  =[False, False, False, False, False, False]
lV_sign_mod  = [False, False, False, False, False, False]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = False

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod)
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod)
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif)


#### Compute delta_m

In [ ]:
delta_m_a1 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case A2

#### Atmosphere profil

In [ ]:
mol_sca = np.array([0., 0.1])[None,:]
mol_abs= np.array([0., 0.])[None,:]
z = np.array([1., 0.])
wl = 550.
proA2 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs).calc(wl)
surfA2  = LambSurface(ALB=Albedo_cst(0.3))
NBPH = 1e8

#### Compute radiances

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

SZA = 50.
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mA2F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=550., NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proA2, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA2, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mA2F.save(output_folder_path + "iprt_a2_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances (Forward, U must be multiplied by -1)
m = mA2F
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a2_smartg_ref.dat"
case_name = "A2"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]
lSZA      = [50., 50.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA, lVZA=[180.-VZA, VZA],
                         lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename  = output_folder_path + "iprt_case_a2_smartg_ref.dat"
case_name = "A2"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]
lSZA      = [50., 50.]
lSAA      = [0., 0.]

smartg_a2 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a2 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a2_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_a2
ref_model = mystic_a2
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[False, False]
lV_sign_mod  = [False, False]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod)
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod)
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif)

#### Compute delta_m

In [ ]:
delta_m_a2 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case A3

#### Atmosphere profil

In [ ]:
# z profil
z = np.array([1., 0.])
nz = len(z[1:])

# molecular scattering and absorption
mol_sca = np.array([0., 0.])[None,:]
mol_abs= np.array([0., 0.])[None,:]

# aerosol extinction and single scattering albedo
aer_tau_ext = np.full_like(mol_sca, 0.2, dtype=np.float32)
aer_tau_ext[:,0] = 0. # dtau TOA equal to 0
aer_ssa = np.full_like(mol_sca, 0.975683, dtype=np.float32)
prof_aer = (aer_tau_ext, aer_ssa)

# aerosol phase matrix
NTH = 18001
theta = np.linspace(0, 180, NTH)
wl = np.array([350.])
nwl = len(wl)
file_aer_phase = OPT_PROP_PATH + "waso.mie.cdf"
aer_phase = read_phase_nth_cte(filename=file_aer_phase, nb_theta=NTH, normalize=True)
nstk = aer_phase.shape[2]

aer_pha = np.zeros((nwl, nz, nstk, NTH), dtype=np.float32)
# Same phase for all altitude (here only one)
for iz in range (0, nz):
    aer_pha[:,iz,:,:] = aer_phase.sub()[:,0,:,:].sub({'wav_phase':Idx(wl)}).sub({'theta_atm':Idx(theta)}).data
aer_phase = LUT(aer_pha, axes=[wl, z[1:], None, theta], names=['wav', 'z', 'stk', 'theta'])
pha_atm, ipha_atm = calc_iphase(aer_phase, np.array([wl]), z)
lpha_lut = []
for i in range (0, pha_atm.shape[0]):
    lpha_lut.append(LUT(pha_atm[i,:,:], axes=[None, np.linspace(0, 180, NTH)], names=['stk', 'theta_atm'])) 

# atmosphere profil
proA3 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs, prof_aer=prof_aer, prof_phases=(ipha_atm, lpha_lut)).calc(wl, phase=False)

# ground surface
surfA3  = None
NBPH = 1e8

#### Compute radiances

In [ ]:
SZA = 40.
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6 # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

mA3F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), NF=NTH,atm=proA3, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA3, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.0, stdev=True, SEED=SEED*2)

In [ ]:
mA3F.save(output_folder_path + "iprt_a3_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances (Forward, U must be multiplied by -1)
m = mA3F
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a3_smartg_ref.dat"
case_name = "A3"
ldepol    = [0.0, 0.0]
lalt      = [0., 1.]
lSZA      = [40., 40.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA, lVZA=[VZA, VZA],
                         lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_a3_smartg_ref.dat"
case_name    = "A3"
ldepol       = [0.0, 0.0]
lalt         = [0., 1.]
lSZA         = [40., 40.]
lSAA         = [0., 0.]

smartg_a3 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a3 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a3_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_a3
ref_model = mystic_a3
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [True, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod)
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod)
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif)

#### Compute delta_m

In [ ]:
delta_m_a3 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case A4

#### Atmosphere profil

In [ ]:
# z profil
z = np.array([1., 0.])
nz = len(z[1:])

# molecular scattering and absorption
mol_sca = np.array([0., 0.])[None,:]
mol_abs= np.array([0., 0.])[None,:]

# aerosol extinction and single scattering albedo
aer_tau_ext = np.full_like(mol_sca, 0.2, dtype=np.float32)
aer_tau_ext[:,0] = 0. # dtau TOA equal to 0
aer_ssa = np.full_like(mol_sca, 0.787581, dtype=np.float32)
prof_aer = (aer_tau_ext, aer_ssa)

# aerosol phase matrix
NTH = 180001
theta = np.linspace(0, 180, NTH)
wl = np.array([350.])
nwl = len(wl)

file_aer_phase = OPT_PROP_PATH + "sizedistr_spheroid.cdf"
aer_phase = read_phase_nth_cte(filename=file_aer_phase, nb_theta=NTH, normalize=True)
nstk = aer_phase.shape[2]
aer_pha = np.zeros((nwl, nz, nstk, NTH), dtype=np.float32)
# Same phase for all altitude (here only one)
for iz in range (0, nz):
    aer_pha[:,iz,:,:] = aer_phase.sub()[:,0,:,:].sub({'wav_phase':Idx(wl)}).sub({'theta_atm':Idx(theta)}).data
aer_phase = LUT(aer_pha, axes=[wl, z[1:], None, theta], names=['wav', 'z', 'stk', 'theta'])

pha_atm, ipha_atm = calc_iphase(aer_phase, np.array([wl]), z)
lpha_lut = []
for i in range (0, pha_atm.shape[0]):
    lpha_lut.append(LUT(pha_atm[i,:,:], axes=[None, np.linspace(0, 180, NTH)], names=['stk', 'theta_atm'])) 

# atmosphere profil
proA4 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs, prof_aer=prof_aer, prof_phases=(ipha_atm, lpha_lut)).calc(wl, phase=False)

# ground surface
surfA4  = None
NBPH = 1e8

#### Compute radiances

In [ ]:
SZA = 40.
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6 # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

mA4F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/100.)), NF=NTH,atm=proA4, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.0, stdev=True, SEED=SEED)

In [ ]:
mA4F.save(output_folder_path + "iprt_a4_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances (Forward, U must be multiplied by -1)
m = mA4F
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a4_smartg_ref.dat"
case_name = "A4"
ldepol    = [0.0, 0.0]
lalt      = [0., 1.]
lSZA      = [40., 40.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA, lVZA=[VZA, VZA],
                         lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_a4_smartg_ref.dat"
case_name    = "A4"
ldepol       = [0.0, 0.0]
lalt         = [0., 1.]
lSZA         = [40., 40.]
lSAA         = [0., 0.]

smartg_a4 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a4 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a4_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_a4
ref_model = mystic_a4
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [True, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod)
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod)
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif)

#### Compute delta_m

In [ ]:
delta_m_a4 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case A5

#### Atmosphere profil

In [ ]:
# z profil
z = np.array([1., 0.])
nz = len(z[1:])

# molecular scattering and absorption
mol_sca = np.array([0., 0.])[None,:]
mol_abs= np.array([0., 0.])[None,:]

# cloud extinction and single scattering albedo
cld_tau_ext = np.full_like(mol_sca, 5., dtype=np.float32)
cld_tau_ext[:,0] = 0. # dtau TOA equal to 0
cld_ssa = np.full_like(mol_sca, 0.999979, dtype=np.float32)
prof_aer = (cld_tau_ext, cld_ssa)

# cloud phase matrix
NTH = 18001
wl = np.array([800.])
nwl = len(wl)
theta = np.linspace(0, 180, NTH)
file_cld_phase = OPT_PROP_PATH + "watercloud.mie.cdf"
cld_phase = read_phase_nth_cte(filename=file_cld_phase, nb_theta=NTH, normalize=True)
nstk = cld_phase.shape[2]

cld_pha = np.zeros((nwl, nz, nstk, NTH), dtype=np.float32)
# Same phase for all altitude (here only one)
for iz in range (0, nz):
    cld_pha[:,iz,:,:] = cld_phase.sub()[:,0,:,:].sub({'wav_phase':Idx(wl)}).sub({'theta_atm':Idx(theta)}).data
cld_phase = LUT(cld_pha, axes=[wl, z[1:], None, theta], names=['wav', 'z', 'stk', 'theta'])

pha_atm, ipha_atm = calc_iphase(cld_phase, np.array([wl]), z)
lpha_lut = []
for i in range (0, pha_atm.shape[0]):
    lpha_lut.append(LUT(pha_atm[i,:,:], axes=[None, theta], names=['stk', 'theta_atm']))  

# atmosphere profil
proA5 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs, prof_aer=prof_aer, prof_phases=(ipha_atm, lpha_lut)).calc(wl, phase=False)

# ground surface
surfA5  = None
NBPH = 1e8

#### Compute radiances (principal plane)

In [ ]:
SZA = 50.
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

VZAMIN = 100.
VZAMAX = 180.
VZAINC = 1.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAA = np.array([0., 180.])

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6 # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

mA5F_pp = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), NF=NTH,atm=proA5, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA5, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mA5F_pp.save(output_folder_path + "iprt_a5_smartg_pp_ref.nc", overwrite=True)

#### Convert into iprt output format (principal plane)

In [ ]:
# Radiances (Forward, U must be multiplied by -1)
m = mA5F_pp
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a5_smartg_pp_ref.dat"
case_name = "A5_pp"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]
lSZA      = [50., 50.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA, lVZA=[VZA, VZA],
                         lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC (principal plane)

In [ ]:
filename  = output_folder_path + "iprt_case_a5_smartg_pp_ref.dat"
case_name = "A5_pp"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]

smartg_a5_pp = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a5_pp = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a5_pp_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model        = smartg_a5_pp
ref_model    = mystic_a5_pp
mod_name     = "SMARTG"
rmod_name    = "MYSTIC"

lINVTH_rmod  = [True, True]
lINVTH_mod   = [True, True]
lU_sign_rmod  =[False, False]
lU_sign_mod  = [False, False]

lIQUVyMin = [[0., -3e-3, -1.5e-4, -2e-5], [0., -2e-2, -1.2e-4, -1e-5]]
lIQUVyMax = [[3.5, 4e-3, 2e-4, 3e-5], [2.5e-1, 1.5e-2, 6e-5, 1e-5]]

quantity = ['transmittance', 'reflectance']

NBRES = len(ldepol)
for ires in range (0,NBRES):
    I_rmod, Q_rmod, U_rmod, V_rmod, Istd_rmod, Qstd_rmod, Ustd_rmod, Vstd_rmod = seclect_iprt_IQUV(ref_model, lalt[ires], change_U_sign=lU_sign_rmod[ires], inv_thetas=lINVTH_rmod[ires], I_index=5, va_index=3, phi_index=4, z_index=0, stdev=True)
    I_mod, Q_mod, U_mod, V_mod, Istd_mod, Qstd_mod, Ustd_mod, Vstd_mod = seclect_iprt_IQUV(model, lalt[ires], change_U_sign=lU_sign_mod[ires], inv_thetas=lINVTH_mod[ires], stdev=True)

    VZA = np.unique(model[:,4])
    VZAn = np.sort(np.concatenate((VZA-180, 180-VZA)))
    NVZA = round(I_mod.shape[0]*2)

    # ref_model IQUV and stdev IQUV
    IQUV_pp_rmod = np.zeros((4,NVZA), dtype=np.float32)
    IQUV_pp_rmod[0,:]=np.concatenate((I_rmod[:,1], I_rmod[::-1,0]))
    IQUV_pp_rmod[1,:]=np.concatenate((Q_rmod[:,1], Q_rmod[::-1,0]))
    IQUV_pp_rmod[2,:]=np.concatenate((U_rmod[:,1], U_rmod[::-1,0]))
    IQUV_pp_rmod[3,:]=np.concatenate((V_rmod[:,1], V_rmod[::-1,0]))
    IQUVstd_pp_rmod = np.zeros((4,NVZA), dtype=np.float32)
    IQUVstd_pp_rmod[0,:]=np.concatenate((Istd_rmod[:,1], Istd_rmod[::-1,0]))
    IQUVstd_pp_rmod[1,:]=np.concatenate((Qstd_rmod[:,1], Qstd_rmod[::-1,0]))
    IQUVstd_pp_rmod[2,:]=np.concatenate((Ustd_rmod[:,1], Ustd_rmod[::-1,0]))
    IQUVstd_pp_rmod[3,:]=np.concatenate((Vstd_rmod[:,1], Vstd_rmod[::-1,0]))

    # model IQUV and stdev IQUV
    IQUV_pp_mod = np.zeros((4,NVZA), dtype=np.float32)
    IQUV_pp_mod[0,:]=np.concatenate((I_mod[:,1], I_mod[::-1,0]))
    IQUV_pp_mod[1,:]=np.concatenate((Q_mod[:,1], Q_mod[::-1,0]))
    IQUV_pp_mod[2,:]=np.concatenate((U_mod[:,1], U_mod[::-1,0]))
    IQUV_pp_mod[3,:]=np.concatenate((V_mod[:,1], V_mod[::-1,0]))
    IQUVstd_pp_mod = np.zeros((4,NVZA), dtype=np.float32)
    IQUVstd_pp_mod[0,:]=np.concatenate((Istd_mod[:,1], Istd_mod[::-1,0]))
    IQUVstd_pp_mod[1,:]=np.concatenate((Qstd_mod[:,1], Qstd_mod[::-1,0]))
    IQUVstd_pp_mod[2,:]=np.concatenate((Ustd_mod[:,1], Ustd_mod[::-1,0]))
    IQUVstd_pp_mod[3,:]=np.concatenate((Vstd_mod[:,1], Vstd_mod[::-1,0]))

    plot_iprt_radiances(IQUV_obs=IQUV_pp_rmod, IQUV_mod=IQUV_pp_mod, IQUVstd_obs=IQUVstd_pp_rmod, IQUVstd_mod=IQUVstd_pp_mod, xaxis=VZAn,
                        xlabel= 'VZA [deg]', IQUVyMin=lIQUVyMin[ires], IQUVyMax=lIQUVyMax[ires], title=f'{quantity[ires]}  {rmod_name}-red {mod_name}-blue')
    
    if (ires == 0):
        IQUV_pp_mod_tot = IQUV_pp_mod.copy()
        IQUV_pp_rmod_tot = IQUV_pp_rmod.copy()
    else:
        IQUV_pp_mod_tot = np.concatenate((IQUV_pp_mod_tot, IQUV_pp_mod), axis=1)
        IQUV_pp_rmod_tot = np.concatenate((IQUV_pp_rmod_tot, IQUV_pp_rmod), axis=1)


#### Compute delta_m (principal plane)

In [ ]:
delta_m_a5_pp = compute_deltam(IQUV_pp_rmod_tot, IQUV_pp_mod_tot)

#### Compute radiances (almucantar)

In [ ]:
SZA = 50.
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

VZA = np.array([130.])

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 1.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6 # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}

mA5F_al = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), NF=NTH,atm=proA5, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA5, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mA5F_al.save(output_folder_path + "iprt_a5_smartg_al_ref.nc", overwrite=True)

#### Convert into iprt output format (almucantar)

In [ ]:
# Radiances (Forward, U must be multiplied by -1)
m = mA5F_al
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a5_smartg_al_ref.dat"
case_name = "A5_al"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]
lSZA      = [50., 50.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA, lVZA=[VZA, VZA],
                         lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC (almucantar)

In [ ]:
filename  = output_folder_path + "iprt_case_a5_smartg_al_ref.dat"
case_name = "A5_al"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]

smartg_a5_al = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a5_al = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a5_al_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model        = smartg_a5_al
ref_model    = mystic_a5_al
mod_name     = "SMARTG"
rmod_name    = "MYSTIC"

lINVTH_rmod  = [True, True]
lINVTH_mod   = [True, True]
lU_sign_rmod  =[False, False]
lU_sign_mod  = [False, False]

lIQUVyMin = [[0., -3.5e-3, -3e-3, -1.5e-5], [6e-2, -1e-2, -2e-3, -5e-5]]
lIQUVyMax = [[3.5, 5e-4, 5e-4, 2.5e-5], [1.2e-1, 2e-2, 1.2e-2, 2e-5]]

quantity = ['transmittance', 'reflectance']

NBRES = len(ldepol)
for ires in range (0,NBRES):
    I_rmod, Q_rmod, U_rmod, V_rmod, Istd_rmod, Qstd_rmod, Ustd_rmod, Vstd_rmod = seclect_iprt_IQUV(ref_model, lalt[ires], change_U_sign=lU_sign_rmod[ires], inv_thetas=lINVTH_rmod[ires], I_index=5, va_index=3, phi_index=4, z_index=0, stdev=True)
    I_mod, Q_mod, U_mod, V_mod, Istd_mod, Qstd_mod, Ustd_mod, Vstd_mod = seclect_iprt_IQUV(model, lalt[ires], change_U_sign=lU_sign_mod[ires], inv_thetas=lINVTH_mod[ires], stdev=True)

    VAA = np.unique(smartg_a5_al[:,5])
    VAAn = VAA
    NVAA = round(I_mod.shape[1])

    # ref_model IQUV and stdev IQUV
    IQUV_al_rmod = np.zeros((4,NVAA), dtype=np.float32)
    IQUV_al_rmod[0,:]=I_rmod[0,:]
    IQUV_al_rmod[1,:]=Q_rmod[0,:]
    IQUV_al_rmod[2,:]=U_rmod[0,:]
    IQUV_al_rmod[3,:]=V_rmod[0,:]
    IQUVstd_al_rmod = np.zeros((4,NVAA), dtype=np.float32)
    IQUVstd_al_rmod[0,:]=Istd_rmod[0,:]
    IQUVstd_al_rmod[1,:]=Qstd_rmod[0,:]
    IQUVstd_al_rmod[2,:]=Ustd_rmod[0,:]
    IQUVstd_al_rmod[3,:]=Vstd_rmod[0,:]

    # model IQUV and stdev IQUV
    IQUV_al_mod = np.zeros((4,NVAA), dtype=np.float32)
    IQUV_al_mod[0,:]=I_mod[0,:]
    IQUV_al_mod[1,:]=Q_mod[0,:]
    IQUV_al_mod[2,:]=U_mod[0,:]
    IQUV_al_mod[3,:]=V_mod[0,:]
    IQUVstd_al_mod = np.zeros((4,NVAA), dtype=np.float32)
    IQUVstd_al_mod[0,:]=Istd_mod[0,:]
    IQUVstd_al_mod[1,:]=Qstd_mod[0,:]
    IQUVstd_al_mod[2,:]=Ustd_mod[0,:]
    IQUVstd_al_mod[3,:]=Vstd_mod[0,:]

    plot_iprt_radiances(IQUV_obs=IQUV_al_rmod, IQUV_mod=IQUV_al_mod, IQUVstd_obs=IQUVstd_al_rmod, IQUVstd_mod=IQUVstd_al_mod, xaxis=VAAn,
                        xlabel= 'VZA [deg]', IQUVyMin=lIQUVyMin[ires], IQUVyMax=lIQUVyMax[ires], title=f'{quantity[ires]}  {rmod_name}-red {mod_name}-blue')
    
    if (ires == 0):
        IQUV_al_mod_tot = IQUV_al_mod.copy()
        IQUV_al_rmod_tot = IQUV_al_rmod.copy()
    else:
        IQUV_al_mod_tot = np.concatenate((IQUV_al_mod_tot, IQUV_al_mod), axis=1)
        IQUV_al_rmod_tot = np.concatenate((IQUV_al_rmod_tot, IQUV_al_rmod), axis=1)

#### Compute delta_m (almucantar)

In [ ]:
delta_m_a5_al = compute_deltam(IQUV_al_rmod_tot, IQUV_al_mod_tot)

### Case A6

#### Atmosphere profil

In [ ]:
mol_sca = np.array([0., 0.1])[None,:]
mol_abs= np.array([0., 0.])[None,:]
z = np.array([1., 0.])
wl = 550.
proA6 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs).calc(550.)
surfA6  = RoughSurface(WIND=2., BRDF=True, WAVE_SHADOW=True, NH2O=1.33)
NBPH = 1e8

#### Compute radiances

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

SZA = 45.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mA6F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proA6, OUTPUT_LAYERS=int(1),
                le=le, surf=surfA6, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mA6F.save(output_folder_path + "iprt_a6_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances 
VZA = 180.-mA6F.axes['Zenith angles']
VAA = -mA6F.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_a6_smartg_ref.dat"
case_name = "A6"
ldepol    = [0.03, 0.03]
lalt      = [0., 1.]
lSZA      = [45., 45.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[mA6F, mA6F], lU_sign=[-1, -1], case_name="A6", ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_a6_smartg_ref.dat"
case_name    = "A6"
ldepol       = [0.03, 0.03]
lalt         = [0., 1.]
lSZA         = [45., 45.]
lSAA         = [0., 0.]

smartg_a6 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_a6 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_a6_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_a6
ref_model = mystic_a6
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod)
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod)
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif)

#### compute delta_m

In [ ]:
delta_m_a6 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

## Test cases with realistic atmospheric profiles

### Case B1
- In the paper and plots are not considered VZA = 95 deg (30km) and 85 deg (0km),
mistake in the description at (https://www.meteo.physik.uni-muenchen.de/~iprt/doku.php?id=intercomparisons:b1_rayleigh).

#### Atmosphere profil

In [ ]:
mol_sca_filename  = OPT_PROP_PATH + "tau_rayleigh_450.dat"
wl = 450.
z = np.squeeze(pd.read_csv(mol_sca_filename, header=None, usecols=[0], dtype=float, skiprows=1, sep=r'\s+').values)
ZS = len(z)
sca = pd.read_csv(mol_sca_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)
proB1 = AtmAFGL('afglt', grid=z, prof_ray=sca, prof_abs=np.zeros_like(sca)).calc(wl)
surfB1 = None
NBPH = 1e8

#### Compute radiances at 0 and 30 km (BOA and TOA)

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}#, 'zip':True}

SZA = 60.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mB1F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proB1, OUTPUT_LAYERS=int(1),
                le=le, surf=surfB1, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mB1F.save(output_folder_path + "iprt_b1_smartg_ref.nc", overwrite=True)

#### Convert into iprt output form

In [ ]:
# Radiances 
m = mB1F
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_b1_smartg_ref.dat"
case_name = "B1"
ldepol    = [0.03, 0.03]
lalt      = [0., 30.]
lSZA      = [60., 60.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_b1_smartg_ref.dat"
case_name    = "B1"
ldepol       = [0.03, 0.03]
lalt         = [0., 30.]
lSZA         = [60., 60.]
lSAA         = [0., 0.]
lVZA         = [np.round(180.-VZA), np.round(VZA)]

smartg_b1 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_b1 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_b1_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_b1
ref_model = mystic_b1
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod, thetas=lVZA[ires])
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod, thetas=lVZA[ires])
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=ldepol[ires], title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif, thetas=lVZA[ires])

#### Compute delta_m

In [ ]:
delta_m_b1 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case B2

#### Atmosphere profil

In [ ]:
mol_sca_filename  = OPT_PROP_PATH + "tau_rayleigh_325.dat"
mol_abs_filename  = OPT_PROP_PATH + "tau_molabs_325.dat"
wl = 325.
z = np.squeeze(pd.read_csv(mol_sca_filename, header=None, usecols=[0], dtype=float, skiprows=1, sep=r'\s+').values)
ZS = len(z)
sca = pd.read_csv(mol_sca_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)
abs = pd.read_csv(mol_abs_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)
proB2 = AtmAFGL('afglt', grid=z, prof_ray=sca, prof_abs=abs).calc(wl)
surfB2 = None
NBPH = 1e8

#### Compute radiances at 0 and 30 km

In [ ]:
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}

SZA = 60.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mB2F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proB2, OUTPUT_LAYERS=int(1),
                le=le, surf=surfB2, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mB2F.save(output_folder_path + "iprt_b2_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
m = mB2F
# Radiances
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_b2_smartg_ref.dat"
case_name = "B2"
ldepol    = [0.03, 0.03]
lalt      = [0., 30.]
lSZA      = [60., 60.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_b2_smartg_ref.dat"
case_name    = "B2"
ldepol       = [0.03, 0.03]
lalt         = [0., 30.]
lSZA         = [60., 60.]
lSAA         = [0., 0.]
lVZA         = [np.round(180.-VZA), np.round(VZA)]

smartg_b2 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_b2 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_b2_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_b2
ref_model = mystic_b2
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    # !!!! mistake in MYSTIC depol, depol is set to 0 instead of 0.03 !!!!
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod, thetas=lVZA[ires])
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod, thetas=lVZA[ires])
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif, thetas=lVZA[ires])

#### Compute delta_m

In [ ]:
delta_m_b2 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case B3

#### Atmosphere profil

In [ ]:
# molecular scattering and absorption
mol_sca_filename  = OPT_PROP_PATH + "tau_rayleigh_350.dat"
mol_abs_filename  = OPT_PROP_PATH + "tau_molabs_350.dat"
wl = np.array([350.])
nwl= len(wl)
z = np.squeeze(pd.read_csv(mol_sca_filename, header=None, usecols=[0], dtype=float, skiprows=1, sep=r'\s+').values)
ZS = len(z)
nz = len(z[1:])
mol_sca = pd.read_csv(mol_sca_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)
mol_abs = pd.read_csv(mol_abs_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)

# aerosol extinction and single scattering albedo
aer_ext_filename  = OPT_PROP_PATH + "tau_aerosol.dat"
aer_tau_ext  = pd.read_csv(aer_ext_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)
aer_ssa  = np.full_like(aer_tau_ext, 0.787581)
prof_aer = (aer_tau_ext, aer_ssa)

# aerosol phase matrix
NTH = 18001 
theta = np.linspace(0, 180, NTH)
file_aer_phase = OPT_PROP_PATH + "sizedistr_spheroid.cdf"
aer_phase = read_phase_nth_cte(filename=file_aer_phase, nb_theta=NTH, normalize=True)
nstk = aer_phase.shape[2]
aer_pha = np.zeros((nwl, nz, nstk, NTH), dtype=np.float32)
# Same phase for all altitude (here only one)
for iz in range (0, nz):
    aer_pha[:,iz,:,:] = aer_phase.sub()[:,0,:,:].sub({'wav_phase':Idx(wl)}).sub({'theta_atm':Idx(theta)}).data
aer_phase = LUT(aer_pha, axes=[wl, z[1:], None, theta], names=['wav', 'z', 'stk', 'theta'])
pha_atm, ipha_atm = calc_iphase(aer_phase, wl, z)
lpha_lut = []
for i in range (0, pha_atm.shape[0]):
    lpha_lut.append(LUT(pha_atm[i,:,:], axes=[None, theta], names=['stk', 'theta_atm'])) 

# atmosphere profil
proB3 = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=mol_abs, prof_aer=prof_aer, prof_phases=(ipha_atm, lpha_lut)).calc(wl, phase=False)
surfB3 = None
NBPH = 1e8

#### Compute radiances

In [ ]:
VZAMIN = 100
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}

SZA = 30.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mB3F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proB3, OUTPUT_LAYERS=int(1),
                le=le, surf=surfB3, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED)

In [ ]:
mB3F.save(output_folder_path + "iprt_b3_smartg_ref.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
m = mB3F
# Radiances
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_b3_smartg_ref.dat"
case_name = "B3"
ldepol    = [0.03, 0.03]
lalt      = [0., 30.]
lSZA      = [30., 30.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_b3_smartg_ref.dat"
case_name    = "B3"
ldepol       = [0.03, 0.03]
lalt         = [0., 30.]
lSZA         = [30., 30.]
lSAA         = [0., 0.]
lVZA         = [np.round(180.-VZA), np.round(VZA)]

smartg_b3 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_b3 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_b3_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_b3
ref_model = mystic_b3
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = True
avoidP_dif  = False
sym = True

NBRES = len(ldepol)
for ires in range (0,NBRES):
    # !!!! mistake in MYSTIC depol, depol is set to 0 instead of 0.03 !!!!
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod, thetas=lVZA[ires])
    
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
    I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                        inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod, thetas=lVZA[ires])
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif, thetas=lVZA[ires])

#### Compute delta_m

In [ ]:
delta_m_b3 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

### Case B4

#### Atmosphere profil

In [ ]:
# molecular scattering, no absorption
mol_sca_filename  = OPT_PROP_PATH + "tau_rayleigh_800.dat"
wl = np.array([800.])
nwl = len(wl)
z = np.squeeze(pd.read_csv(mol_sca_filename, header=None, usecols=[0], dtype=float, skiprows=1, sep=r'\s+').values)
ZS = len(z)
nz = len(z[1:])
mol_sca = pd.read_csv(mol_sca_filename, header=None, usecols=[1], dtype=float, skiprows=1, sep=r'\s+').values.reshape(1,ZS)

# cloud extinction and single scattering albedo
COT                         = 5 # cloud optical thickness
cld_zmin                    = 2 # cloud zmin
cld_zmax                    = 3 # cloud zmax
cld_tau_ext                 = np.zeros((1, len(z)), dtype='float32') # len(wl), len(dZ)
z_bis                       = np.array([cld_zmax, cld_zmax, cld_zmin, cld_zmin, 0.], dtype='float32')
densities                   = np.array([0., 1., 1., 0., 0.], dtype='float32')[:,None]
dZ                          = -diff1(z)
dens                        = trapzinterp(densities[:,0], z_bis, z)
cld_tau_ext[0,:]            = COT * dens * dZ
cld_tau_ext[cld_tau_ext<=0] = 0
cld_ssa                     = np.full_like(cld_tau_ext, 0.999979)
cld_ssa[cld_tau_ext==0]     = 1
prof_aer                    = (cld_tau_ext, cld_ssa)

# cloud phase matrix
NTH = 18001  # The water cloud has a phase function with a non-negligible peak, then a sufficiently fine resolution is required.
theta = np.linspace(0, 180, NTH)
file_cld_phase = OPT_PROP_PATH + "watercloud.mie.cdf"
cld_phase = read_phase_nth_cte(filename=file_cld_phase, nb_theta=NTH, normalize=True)
nstk = cld_phase.shape[2]
cld_pha = np.zeros((nwl, nz, nstk, NTH), dtype=np.float32)
# Same phase for all altitude (here only one)
for iz in range (0, nz):
    cld_pha[:,iz,:,:] = cld_phase.sub()[:,0,:,:].sub({'wav_phase':Idx(wl)}).sub({'theta_atm':Idx(theta)}).data
cld_phase = LUT(cld_pha, axes=[wl, z[1:], None, theta], names=['wav', 'z', 'stk', 'theta'])
pha_atm, ipha_atm = calc_iphase(cld_phase, wl, z)
lpha_lut = []
for i in range (0, pha_atm.shape[0]):
    lpha_lut.append(LUT(pha_atm[i,:,:], axes=[None, theta], names=['stk', 'theta_atm'])) 

# atmosphere profil
proB4  = AtmAFGL('afglt', grid=z, prof_ray=mol_sca, prof_abs=np.zeros_like(mol_sca), prof_aer=prof_aer, prof_phases=(ipha_atm, lpha_lut)).calc(wl, phase=False)
surfB4 = RoughSurface(WIND=2., BRDF=True, WAVE_SHADOW=True, NH2O=1.33)
NBPH   = 1e8

#### Compute radiances (0 and 30 km)

In [ ]:
VZAMIN = 100
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

# SMART-G Forward TH and PHI using local estimate (anticlockwise) conversion with VZA and VAA MYSTIC (clockwise)
TH  = 180.-VZA
PHI = -VAA
TH[TH==0] = 1e-6  # avoid problem due to special case of 0
le     = {'th_deg':TH, 'phi_deg':PHI}

SZA = 60.0
SAA = 0.
PHI_0 = 180.-SAA # SMART-G anticlockwise converted to be consistent with MYSTIC

mB4F = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proB4, OUTPUT_LAYERS=int(1),
                le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED, NF=NTH)#, RR=1, WEIGHTRR=0.1)
mB4F2 = S1DF.run(THVDEG=SZA, PHVDEG=PHI_0, wl=wl, NBPHOTONS=NBPH, NBLOOP=min(1e6, round(NBPH/10.)), atm=proB4, OUTPUT_LAYERS=int(1),
                le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, SEED=SEED*2, NF=NTH)

In [ ]:
mB4F.save(output_folder_path + "iprt_b4_smartg_ref.nc", overwrite=True)
mB4F2.save(output_folder_path + "iprt_b4_smartg_ref2.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
m = mB4F
# Radiances
VZA = 180.-m.axes['Zenith angles']
VAA = -m.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_b4_smartg_ref.dat"
filename2  = output_folder_path + "iprt_case_b4_smartg_ref2.dat"
case_name = "B4"
ldepol    = [0.03, 0.03]
lalt      = [0., 30.]
lSZA      = [60., 60.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename, output_layer=['_down (0+)', '_up (TOA)'])
m = mB4F2
convert_SGout_to_IPRTout(lm=[m, m], lU_sign=[-1, -1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                        lVZA=[180.-VZA, VZA], lVAA=[VAA, VAA], file_name=filename2, output_layer=['_down (0+)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_b4_smartg_ref.dat"
case_name    = "B4"
ldepol       = [0.03, 0.03]
lalt         = [0., 30.]
lSZA         = [60., 60.]
lSAA         = [0., 0.]
lVZA         = [np.round(180.-VZA), np.round(VZA)]
lVAA         = [None, None]#[VAA, VAA]

smartg_b4 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_b4 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_b4_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_b4
ref_model = mystic_b4
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [True, True]
avoidP_rmod = False
avoidP_mod  = False
avoidP_dif  = False
sym = True
use_2sim = True

if use_2sim:
    filename2 = output_folder_path + "iprt_case_b4_smartg_ref2.dat"
    model2    = pd.read_csv(filename2, header=None, sep=r'\s+', dtype=float, comment="#").values

NBRES = len(ldepol)
for ires in range (0,NBRES):
    # !!!! mistake in MYSTIC depol, depol is set to 0 instead of 0.03 !!!!
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod, thetas=lVZA[ires], phis=lVAA[ires])
    if not use_2sim:
        title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
        I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                            inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod, thetas=lVZA[ires], phis=lVAA[ires])
    else:
        title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
        I_mod, Q_mod, U_mod, V_mod, Istd_mod, Qstd_mod, Ustd_mod, Vstd_mod = \
            select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                    inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=True, thetas=lVZA[ires], phis=lVAA[ires], outputIQUVstd=True)
        I_mod2, Q_mod2, U_mod2, V_mod2, Istd_mod2, Qstd_mod2, Ustd_mod2, Vstd_mod2 = \
            select_and_plot_polar_iprt(model2, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                    inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=True, thetas=lVZA[ires], phis=lVAA[ires], outputIQUVstd=True)
        I_mod3 = np.zeros_like(I_mod)
        Q_mod3 = np.zeros_like(Q_mod)
        U_mod3 = np.zeros_like(U_mod)
        V_mod3 = np.zeros_like(V_mod)
        for i in range (0, I_mod.shape[0]):
            for j in range (0, I_mod.shape[1]):
                if (Istd_mod[i,j] < Istd_mod2[i,j]): I_mod3[i,j] = I_mod[i,j]
                else: I_mod3[i,j]  = I_mod2[i,j]
                if (Qstd_mod[i,j] < Qstd_mod2[i,j]): Q_mod3[i,j]  = Q_mod[i,j]
                else: Q_mod3[i,j]  = Q_mod2[i,j]
                if (Ustd_mod[i,j] < Ustd_mod2[i,j]): U_mod3[i,j]  = U_mod[i,j]
                else: U_mod3[i,j]  = U_mod2[i,j]
                if (Vstd_mod[i,j] < Vstd_mod2[i,j]): V_mod3[i,j]  = V_mod[i,j]
                else: V_mod3[i,j]  = V_mod2[i,j]
        I_mod = I_mod3.copy()
        Q_mod = Q_mod3.copy()
        U_mod = U_mod3.copy()
        V_mod = V_mod3.copy()
        select_and_plot_polar_iprt(ref_model, z_alti=lalt[0], depol=0., title=title, forceIQUV=[I_mod, Q_mod, U_mod, V_mod], avoid_plot=avoidP_mod, sym=sym, thetas=lVZA[ires], phis=lVAA[ires], 
                                   maxQ=max(np.abs(np.min(Q_rmod)), np.abs(np.max(Q_rmod))), maxU=max(np.abs(np.min(U_rmod)), np.abs(np.max(U_rmod))), maxV=max(np.abs(np.min(V_rmod)), np.abs(np.max(V_rmod))))
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif, thetas=lVZA[ires], phis=lVAA[ires])

#### Compute delta_m

In [ ]:
delta_m_b4 = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)

#### Compute radiances in backward mode (0 km)

In [ ]:
SZA = 60.
SAA = 0.
PHI_0 = 180.-SAA # To follow MYSTIC convention
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([PHI_0])}

# VZA
VZAMIN = 0.
VZAMAX = 80.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

# VAA from 0. to 180.
VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

lsensors = []
NBVZA = len(VZA)
NBVAA = len(VAA)
NBDIR = round(NBVAA*NBVZA)
for iza, za in enumerate(VZA):
    for iaa, aa in enumerate(VAA):
        PHI = -aa+180
        lsensors.append(Sensor(POSZ=0.0, THDEG=za, PHDEG=PHI, LOC='ATMOS')) ### Sensor at 1km altitude

mB4B_0km = S1DB.run(wl=wl, NBPHOTONS=NBPH*NBDIR, NBLOOP=NBPH, atm=proB4, sensor=lsensors, NF=NTH,
                    le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, progress=True, SEED=SEED)

mB4B_0km = mB4B_0km.dropaxis('Azimuth angles', 'Zenith angles')
mB4B_0km.add_axis('Azimuth angles', -VAA+180.)
mB4B_0km.add_axis('Zenith angles', VZA)

for name in mB4B_0km.datasets():
    if 'sensor index' in mB4B_0km[name].names:
        mat_tmp = np.swapaxes(mB4B_0km[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mB4B_0km[name].attrs
        mB4B_0km.rm_lut(name)
        mB4B_0km.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mB4B_0km = mB4B_0km.dropaxis('sensor index')

mB4B_0km2 = S1DB.run(wl=wl, NBPHOTONS=NBPH*NBDIR, NBLOOP=NBPH, atm=proB4, sensor=lsensors, NF=NTH,
                     le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, progress=True, SEED=SEED*2)

mB4B_0km2 = mB4B_0km2.dropaxis('Azimuth angles', 'Zenith angles')
mB4B_0km2.add_axis('Azimuth angles', -VAA+180.)
mB4B_0km2.add_axis('Zenith angles', VZA)

for name in mB4B_0km2.datasets():
    if 'sensor index' in mB4B_0km2[name].names:
        mat_tmp = np.swapaxes(mB4B_0km2[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mB4B_0km2[name].attrs
        mB4B_0km2.rm_lut(name)
        mB4B_0km2.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mB4B_0km2 = mB4B_0km2.dropaxis('sensor index')

In [ ]:
mB4B_0km.save(output_folder_path + "iprt_b4_smartg_ref_BAK_0km.nc", overwrite=True)
mB4B_0km2.save(output_folder_path + "iprt_b4_smartg_ref_BAK_0km2.nc", overwrite=True)

#### Compute radiances in backward mode (30km)

In [ ]:
SZA = 60.
SAA = 0.
PHI_0 = 180.-SAA # To follow MYSTIC convention
le     = {'th_deg':np.array([SZA]), 'phi_deg':np.array([PHI_0])}

# VZA
VZAMIN = 100.
VZAMAX = 180.
VZAINC = 5.
VZA = np.arange(VZAMIN, VZAMAX+VZAINC, VZAINC)

# VAA from 0. to 180.
VAAMIN = 0.
VAAMAX = 180.
VAAINC = 5.
VAA = np.arange(VAAMIN, VAAMAX+VAAINC, VAAINC)

lsensors = []
NBVZA = len(VZA)
NBVAA = len(VAA)
NBDIR = round(NBVAA*NBVZA)
for iza, za in enumerate(VZA):
    for iaa, aa in enumerate(VAA):
        PHI = -aa+180
        lsensors.append(Sensor(POSZ=30.0, THDEG=za, PHDEG=PHI, LOC='ATMOS')) ### Sensor at 1km altitude

mB4B_30km = S1DB.run(wl=wl, NBPHOTONS=NBPH*NBDIR, NBLOOP=NBPH, atm=proB4, sensor=lsensors, NF=NTH,
                     le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, progress=True, SEED=SEED)

mB4B_30km = mB4B_30km.dropaxis('Azimuth angles', 'Zenith angles')
mB4B_30km.add_axis('Azimuth angles', -VAA+180.)
mB4B_30km.add_axis('Zenith angles', VZA)

for name in mB4B_30km.datasets():
    if 'sensor index' in mB4B_30km[name].names:
        mat_tmp = np.swapaxes(mB4B_30km[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mB4B_30km[name].attrs
        mB4B_30km.rm_lut(name)
        mB4B_30km.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mB4B_30km = mB4B_30km.dropaxis('sensor index')

mB4B_30km2 = S1DB.run(wl=wl, NBPHOTONS=NBPH*NBDIR, NBLOOP=NBPH, atm=proB4, sensor=lsensors, NF=NTH,
                      le=le, surf=surfB4, XBLOCK = 64, XGRID = 1024, BEER=1, DEPO=0.03, stdev=True, progress=True, SEED=SEED*2)

mB4B_30km2 = mB4B_30km2.dropaxis('Azimuth angles', 'Zenith angles')
mB4B_30km2.add_axis('Azimuth angles', -VAA+180.)
mB4B_30km2.add_axis('Zenith angles', VZA)

for name in mB4B_30km2.datasets():
    if 'sensor index' in mB4B_30km2[name].names:
        mat_tmp = np.swapaxes(mB4B_30km2[name][:].reshape(len(VZA), len(VAA)), 0, 1)
        attrs_tmp = mB4B_30km2[name].attrs
        mB4B_30km2.rm_lut(name)
        mB4B_30km2.add_dataset(name, mat_tmp, ['Azimuth angles', 'Zenith angles'], attrs=attrs_tmp)

mB4B_30km2 = mB4B_30km2.dropaxis('sensor index')

In [ ]:
mB4B_30km.save(output_folder_path + "iprt_b4_smartg_ref_BAK_30km.nc", overwrite=True)
mB4B_30km2.save(output_folder_path + "iprt_b4_smartg_ref_BAK_30km2.nc", overwrite=True)

#### Convert into iprt output format

In [ ]:
# Radiances
VZA_0km = mB4B_0km.axes['Zenith angles']
VAA_0km = 180-mB4B_0km.axes['Azimuth angles']

VZA_30km = mB4B_30km.axes['Zenith angles']
VAA_30km = 180.-mB4B_30km.axes['Azimuth angles']

filename  = output_folder_path + "iprt_case_b4_smartg_BAK_ref.dat"
filename2  = output_folder_path + "iprt_case_b4_smartg_BAK_ref2.dat"
case_name = "B4"
ldepol    = [0.03, 0.03]
lalt      = [0., 30.]
lSZA      = [60., 60.]
lSAA      = [0., 0.]

# convert
convert_SGout_to_IPRTout(lm=[mB4B_0km, mB4B_30km], lU_sign=[1, 1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                         lVZA=[VZA_0km, VZA_30km], lVAA=[VAA_0km, VAA_30km], file_name=filename, output_layer=['_up (TOA)', '_up (TOA)'])

convert_SGout_to_IPRTout(lm=[mB4B_0km2, mB4B_30km2], lU_sign=[1, 1], case_name=case_name, ldepol=ldepol, lalt=lalt, lSZA=lSZA, lSAA=lSAA,
                         lVZA=[VZA_0km, VZA_30km], lVAA=[VAA_0km, VAA_30km], file_name=filename2, output_layer=['_up (TOA)', '_up (TOA)'])

#### Comparison with MYSTIC

In [ ]:
filename     = output_folder_path + "iprt_case_b4_smartg_BAK_ref.dat"
case_name    = "B4"
ldepol       = [0.03, 0.03]
lalt         = [0., 30.]
lSZA         = [60., 60.]
lSAA         = [0., 0.]
lVZA         = [np.round(180.-VZA_30km), np.round(VZA_30km)]
lVAA         = [None, None]#[VAA, VAA]

smartg_b4 = pd.read_csv(filename, header=None, sep=r'\s+', dtype=float, comment="#").values
mystic_b4 = pd.read_csv(MYSTIC_RES_PATH + "iprt_case_b4_mystic.dat", header=None, sep=r'\s+', dtype=float, comment="#").values
model     = smartg_b4
ref_model = mystic_b4
mod_name  = "SMARTG"
rmod_name = "MYSTIC"
lINVTH_rmod  = [False, True]
lINVTH_mod   = [False, True]
lU_sign_rmod  =[True, True]
lU_sign_mod  = [True, True]
lV_sign_rmod  =[True, True]
lV_sign_mod  = [False, False]
avoidP_rmod = False
avoidP_mod  = False
avoidP_dif  = False
sym = True
use_2sim = True

if use_2sim:
    filename2 = output_folder_path + "iprt_case_b4_smartg_BAK_ref2.dat"
    model2    = pd.read_csv(filename2, header=None, sep=r'\s+', dtype=float, comment="#").values

NBRES = len(ldepol)
for ires in range (0,NBRES):
    # !!!! mistake in MYSTIC depol, depol is set to 0 instead of 0.03 !!!!
    title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f}  - {lalt[ires]:.0f}km - {rmod_name}"
    I_rmod, Q_rmod, U_rmod, V_rmod = select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, change_U_sign=lU_sign_rmod[ires], change_V_sign=lV_sign_rmod[ires],
                                                                        inv_thetas=lINVTH_rmod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_rmod, thetas=lVZA[ires], phis=lVAA[ires])
    
    
    if not use_2sim:
        title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
        I_mod, Q_mod, U_mod, V_mod = select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                                                inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=avoidP_mod, thetas=lVZA[ires], phis=lVAA[ires],
                                                                maxQ=max(np.abs(np.min(Q_rmod)), np.abs(np.max(Q_rmod))), maxU=max(np.abs(np.min(U_rmod)), np.abs(np.max(U_rmod))), maxV=max(np.abs(np.min(V_rmod)), np.abs(np.max(V_rmod))))
    else:
        title = f"IPRT case {case_name} - depol = {ldepol[ires]} - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {lalt[ires]:.0f}km - {mod_name}"
        I_mod, Q_mod, U_mod, V_mod, Istd_mod, Qstd_mod, Ustd_mod, Vstd_mod = \
            select_and_plot_polar_iprt(model, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                    inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=True, thetas=lVZA[ires], phis=lVAA[ires], outputIQUVstd=True)
        I_mod2, Q_mod2, U_mod2, V_mod2, Istd_mod2, Qstd_mod2, Ustd_mod2, Vstd_mod2 = \
            select_and_plot_polar_iprt(model2, z_alti=lalt[ires], depol=ldepol[ires], title=title, change_U_sign=lU_sign_mod[ires], change_V_sign=lV_sign_mod[ires],
                                    inv_thetas=lINVTH_mod[ires], sym=sym, outputIQUV=True, avoid_plot=True, thetas=lVZA[ires], phis=lVAA[ires], outputIQUVstd=True)
        I_mod3 = np.zeros_like(I_mod)
        Q_mod3 = np.zeros_like(Q_mod)
        U_mod3 = np.zeros_like(U_mod)
        V_mod3 = np.zeros_like(V_mod)
        for i in range (0, I_mod.shape[0]):
            for j in range (0, I_mod.shape[1]):
                if (Istd_mod[i,j] < Istd_mod2[i,j]): I_mod3[i,j] = I_mod[i,j]
                else: I_mod3[i,j]  = I_mod2[i,j]
                if (Qstd_mod[i,j] < Qstd_mod2[i,j]): Q_mod3[i,j]  = Q_mod[i,j]
                else: Q_mod3[i,j]  = Q_mod2[i,j]
                if (Ustd_mod[i,j] < Ustd_mod2[i,j]): U_mod3[i,j]  = U_mod[i,j]
                else: U_mod3[i,j]  = U_mod2[i,j]
                if (Vstd_mod[i,j] < Vstd_mod2[i,j]): V_mod3[i,j]  = V_mod[i,j]
                else: V_mod3[i,j]  = V_mod2[i,j]
        I_mod = I_mod3.copy()
        Q_mod = Q_mod3.copy()
        U_mod = U_mod3.copy()
        V_mod = V_mod3.copy()
        select_and_plot_polar_iprt(ref_model, z_alti=lalt[0], depol=0., title=title, forceIQUV=[I_mod, Q_mod, U_mod, V_mod], avoid_plot=avoidP_mod, sym=sym, thetas=lVZA[ires], phis=lVAA[ires], 
                                   maxQ=max(np.abs(np.min(Q_rmod)), np.abs(np.max(Q_rmod))), maxU=max(np.abs(np.min(U_rmod)), np.abs(np.max(U_rmod))), maxV=max(np.abs(np.min(V_rmod)), np.abs(np.max(V_rmod))))
    
    if (ires == 0):
        IQUV_mod_tot = groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])
        IQUV_rmod_tot = groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])
    else:
        IQUV_mod_tot = np.concatenate((IQUV_mod_tot, groupIQUV(lI=[I_mod], lQ=[Q_mod], lU=[U_mod], lV=[V_mod])), axis=1)
        IQUV_rmod_tot = np.concatenate((IQUV_rmod_tot, groupIQUV(lI=[I_rmod], lQ=[Q_rmod], lU=[U_rmod], lV=[V_rmod])), axis=1)

    I_val = I_rmod-I_mod
    Q_val = Q_rmod-Q_mod
    U_val = U_rmod-U_mod
    V_val = V_rmod-V_mod
    maxI = max(np.abs(np.min(I_val)), np.abs(np.max(I_val)))
    maxQ = max(np.abs(np.min(Q_val)), np.abs(np.max(Q_val)))
    maxU = max(np.abs(np.min(U_val)), np.abs(np.max(U_val)))
    maxV = max(np.abs(np.min(V_val)), np.abs(np.max(V_val)))
    title = f"IPRT case {case_name} - depol = {ldepol[ires]}  - SZA = {lSZA[ires]:.0f} - SAA = {lSAA[ires]:.0f} - {ldepol[ires]:.0f}km - dif ({rmod_name}-{mod_name})"
    select_and_plot_polar_iprt(ref_model, z_alti=lalt[ires], depol=0., title=title, forceIQUV=[I_val, Q_val, U_val, V_val],
                               maxI=maxI, maxQ=maxQ, maxU=maxU, maxV=maxV, cmapI='RdBu_r', avoid_plot=avoidP_dif, thetas=lVZA[ires], phis=lVAA[ires])

#### Compute delta_m

In [ ]:
delta_m_b4_BAK = compute_deltam(obs=IQUV_rmod_tot, mod=IQUV_mod_tot, print_res=True)